In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader as ND2Reader
# import aicsimageio.vendor.omexml as ome
import glob

In [2]:
# channels = ['miRFP670', 'mRuby3','FITC']
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

# for lens    
#     reader.parser._raw_metadata.image_calibration

# import re


def Nd2meta2OMEXML(reader, project=False):
#     from apeer_ometiff_library import omexmlClass
    import aicsimageio.vendor.omexml as omexmlClass
    import re
    
    #Missing TODO:
    #<Image>,  Name = "ImageName"
    #Instrument ID and Detector ID and Objective 
#     frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'wsCameraName']

    # Objective settings with Refractive Index
        #Pixels, 
            #Channel Color = RGB###, EmissionWavelength, Name of Channel.
            #Plane  ExposureTime, Position X, Y, Z (Z is easy as it's in nd2reader metadata)
    def writeplanes(pixel, SizeT=1, SizeZ=1, SizeC=1, order='TZCYX', verbose=False):

        if order == 'TZCYX':

            pixel.DimensionOrder = omexmlClass.DO_XYCZT
            counter = 0
            for t in range(SizeT):
                for z in range(SizeZ):
                    for c in range(SizeC):

                        if verbose:
                            print('Write PlaneTable: ', t, z, c),
                            sys.stdout.flush()

                        pixel.Plane(counter).TheT = t
                        pixel.Plane(counter).TheZ = z
                        pixel.Plane(counter).TheC = c
                        #check basically because of triggered acquisition the arrays shouldn't have the size of "channel"
                        pixel.Plane(counter).DeltaT = reader.get_timesteps()[counter//SizeC]
                        pixel.Plane(counter).PositionZ = nd2meta['z_coordinates'][counter//SizeC]
#                         pixel.Plane(counter).ExposureTime = 
#                         pixel.Plane(counter).PositionX =
#                         pixel.Plane(counter).PositionY = 
#                         pixel.Plane(counter).
                        counter = counter + 1
                        
    
        return pixel
    
    #make a metadata var
    nd2meta = reader.metadata

    extra_meta = reader.parser._raw_metadata.image_text_info[b'SLxImageTextInfo'][b'TextInfoItem_5'].decode()
    extra_meta = re.split(',|;|\r\n',extra_meta)
    extra_dict = dict()
    for line in extra_meta: 
        line = line.strip().strip('- ')
        keyvalue = str.split(line,':') 
        if len(keyvalue) > 1: 
            key = keyvalue[0] 
            value = keyvalue[1] 
            extra_dict[key] = value
#     Series = nd2meta['fields_of_view'][-1]+1
    scalex = nd2meta['pixel_microns']
    scaley = scalex
    
    
    
    if not project:
#         scalez = round(nd2meta['z_coordinates'][1]-nd2meta['z_coordinates'][0],3)
#         scalez = frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep']
        scalez = extra_dict['Step'].split()[0]
    pixeltype = 'uint16'
    dimorder = 'TZCYX'
# print(a)
    omexml = omexmlClass.OMEXML()
#     omexml.image_count = 1
#     omexml.image_count = reader.sizes['v']
    #Try to find if PIMS outputs the filename somehow.
    omexml.image(0).Name = reader.filename
#     for i in range(frames.sizes['t']):
    p = omexml.image(0).Pixels
    p.SizeX = frames.sizes['x']
#     p.SizeX = 2044
    p.SizeY = frames.sizes['y']
    p.SizeC = frames.sizes['c']
    p.SizeT = frames.sizes['t']
    if project:
        p.SizeZ = 1
    else:
        p.SizeZ = frames.sizes['z']
        
    p.PhysicalSizeX = np.float(scalex)
    p.PhysicalSizeY = np.float(scaley)
    if not project:
        p.PhysicalSizeZ = np.float(scalez)
    p.PixelType = pixeltype
    p.channel_count = frames.sizes['c']
    
    if project:
        p.plane_count = 1 * p.SizeT * p.SizeC #* SizeV
    else:
        p.plane_count = p.SizeZ * p.SizeT * p.SizeC #* SizeV


    #I am using separate files for each visit point
    #, if you want one tiff with all visit points (possibly good for panels) 
    #you will need to update this section
    
    if project:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=1, SizeC=p.SizeC, order=dimorder)
    else:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=p.SizeZ, SizeC=p.SizeC, order=dimorder)
    for c in range(p.SizeC):
        p.Channel(c).Name = nd2meta['channels'][c]
        clr = {'miRFP670':  65535 ,'mRuby3' : -16776961,'mRuby' : -16776961, 'a647': 65535,'GFP': 16711935,'FITC': 16711935,'DAPI': 65535}
        p.Channel(c).Color = clr[p.Channel(c).Name]
#         p.Channel(c).EmissionWavelength =
        if pixeltype == 'unit8':
            p.Channel(c).SamplesPerPixel = 1
        if pixeltype == 'unit16':
            p.Channel(c).SamplesPerPixel = 2
            
    p.populate_TiffData()
#     omexml.structured_annotations.add_original_metadata(omexmlClass.OM_SAMPLES_PER_PIXEL, str(p.SizeC))

    return omexml



#########################
def pimsmeta2OMEXML(reader, project=False):
#     from apeer_ometiff_library import omexmlClass
    import aicsimageio.vendor.omexml as omexmlClass
    import re , os
    
    #Missing TODO:
    #<Image>,  Name = "ImageName"
    #Instrument ID and Detector ID and Objective 
#     frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'wsCameraName']

    # Objective settings with Refractive Index
        #Pixels, 
            #Channel Color = RGB###, EmissionWavelength, Name of Channel.
            #Plane  ExposureTime, Position X, Y, Z (Z is easy as it's in nd2reader metadata)
    def writeplanes(pixel, SizeT=1, SizeZ=1, SizeC=1, order='TZCYX', verbose=False):

        if order == 'TZCYX':

            pixel.DimensionOrder = omexmlClass.DO_XYCZT
            counter = 0
            for t in range(SizeT):
                for z in range(SizeZ):
                    for c in range(SizeC):

                        if verbose:
                            print('Write PlaneTable: ', t, z, c),
                            sys.stdout.flush()

                        pixel.Plane(counter).TheT = t
                        pixel.Plane(counter).TheZ = z
                        pixel.Plane(counter).TheC = c
                        #check basically because of triggered acquisition the arrays shouldn't have the size of "channel"
                        pixel.Plane(counter).DeltaT = reader.metadata.PlaneDeltaT(0,counter)
                        pixel.Plane(counter).PositionZ = frames.metadata.PlanePositionZ(0,counter)
                        pixel.Plane(counter).PositionX = frames.metadata.PlanePositionX(0,counter)
                        pixel.Plane(counter).PositionY = frames.metadata.PlanePositionY(0,counter)
#                         pixel.Plane(counter).ExposureTime = 
#                         pixel.Plane(counter).PositionX =
#                         pixel.Plane(counter).PositionY = 
#                         pixel.Plane(counter).
                        counter = counter + 1
                        
    
        return pixel
    
    #make a metadata var
    bfmeta = reader.metadata

    # extra_meta = reader.parser._raw_metadata.image_text_info[b'SLxImageTextInfo'][b'TextInfoItem_5'].decode()
    # extra_meta = re.split(',|;|\r\n',extra_meta)
    # extra_dict = dict()
    # for line in extra_meta: 
        # line = line.strip().strip('- ')
        # keyvalue = str.split(line,':') 
        # if len(keyvalue) > 1: 
            # key = keyvalue[0] 
            # value = keyvalue[1] 
            # extra_dict[key] = value
#     Series = nd2meta['fields_of_view'][-1]+1
    scalex = frames.metadata.PixelsPhysicalSizeX(0)
    scaley = scalex
    
    
    
    if not project:
#         scalez = round(nd2meta['z_coordinates'][1]-nd2meta['z_coordinates'][0],3)
#         scalez = frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep']
        scalez = frames.metadata.PixelsPhysicalSizeZ(0)
    pixeltype = frames.metadata.PixelsType(0)
    # dimorder = 'TZCYX'
    dimorder = frames.metadata.PixelsDimensionOrder(0)
# print(a)
    omexml = omexmlClass.OMEXML()
#     omexml.image_count = 1
#     omexml.image_count = reader.sizes['v']
    #Try to find if PIMS outputs the filename somehow.
    omexml.image(0).Name = os.path.basename(reader.filename)
#     for i in range(frames.sizes['t']):
    p = omexml.image(0).Pixels
    p.SizeX = frames.sizes['x']
#     p.SizeX = 2044
    p.SizeY = frames.sizes['y']
    p.SizeC = frames.sizes['c']
    
    try:
        p.SizeT = frames.sizes['t']
    except: print("Single T")
    
    if project:
        p.SizeZ = 1
    else:
        p.SizeZ = frames.sizes['z']
        
    p.PhysicalSizeX = np.float(scalex)
    p.PhysicalSizeY = np.float(scaley)
    if not project:
        p.PhysicalSizeZ = np.float(scalez)
    p.PixelType = pixeltype
    p.channel_count = frames.sizes['c']
    
    if project:
        p.plane_count = 1 * p.SizeT * p.SizeC #* SizeV
    else:
        p.plane_count = p.SizeZ * p.SizeT * p.SizeC #* SizeV


    #I am using separate files for each visit point
    #, if you want one tiff with all visit points (possibly good for panels) 
    #you will need to update this section
    
    if project:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=1, SizeC=p.SizeC, order=dimorder)
    else:
        p = writeplanes(p, SizeT=p.SizeT, SizeZ=p.SizeZ, SizeC=p.SizeC, order=dimorder)
    for c in range(p.SizeC):
        p.Channel(c).Name = str(frames.metadata.ChannelEmissionWavelength(0,c))
        clr = {'535.0': 16711935, '548.0': 16711935,'626.0' : -16776961 ,'625.0' : -16776961, '670.0':  65535 ,'620.0' : -16776961,
               'miRFP670':  65535 ,'555 nm' : -16776961,'mRuby3' : -16776961,'mRuby' : -16776961, 'a647': 65535,'GFP': 16711935,'FITC': 16711935,'DAPI': 65535,'470 nm': 16711935}
        p.Channel(c).Color = clr[p.Channel(c).Name]
#         p.Channel(c).EmissionWavelength =
        if pixeltype == 'unit8':
            p.Channel(c).SamplesPerPixel = 1
        if pixeltype == 'unit16':
            p.Channel(c).SamplesPerPixel = 2
            
    p.populate_TiffData()
#     omexml.structured_annotations.add_original_metadata(omexmlClass.OM_SAMPLES_PER_PIXEL, str(p.SizeC))

    return omexml




def observer(img, i, *args):
    #mgs.append(img.max(axis=0))
    if i % 5 == 0:
        print('Observing iteration = {} (dtype = {}, max = {:.3f})'.format(i, img.dtype, img.max()))   
#config = tf.ConfigProto(device_count={'GPU': 1})
#algo = fd_restoration.RichardsonLucyDeconvolver(n_dims=acq.data.ndim, pad_min=[1, 1, 1], session_config=config).initialize()



def deconv(chunk):
    # note that algo and cropped_kernel are from global scope ... ugly
#     print("chunk shape", chunk.shape)
    tmp = algo.initialize().run(fd_data.Acquisition(data=chunk, kernel=psf)
#                                 , session_config=tflow.compat.v1.ConfigProto(
#                                 device_count={'GPU': 1}        
#                                 , GPUOptions={'allow_growth' : 4}
#                                         )
                                ,niter=20
                               )
    return tmp.data

In [8]:
# fname = 'igfp1_caruby5_continue001trigger003.nd2'
# dirname = "Y:/IMMUN/Mamede/20200630/*.nd2"
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end

#dirname = "X:/Mamede/Stephanie/20200902/*.nd2"
dirname = "/run/media/jmamede/Joao/SG/f2_igfp_optir3_vitcrutin_plb_n4cl_bili_xy*.ome.tif"
# dirname = "/home/jmamede/Data/pqbp1/20200919FRET/20200902/*R3D.dv"
filelist = glob.glob(dirname)
filelist.sort()
print(filelist)
ficheiro = filelist[0]
frames = pims.bioformats.BioformatsReader(ficheiro,java_memory='1024m')
print(frames.sizes)

metadata = frames.metadata
# print(metadata)

ydivide = 1
xdivide = 2

if xdivide >1:
    depthdivide = (0,0,64)
else:
    depthdivide = (0,0,0)
# print(frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep'])


['/run/media/jmamede/Joao/SG/f2_igfp_optir3_vitcrutin_plb_n4cl_bili_xy21.ome.tif', '/run/media/jmamede/Joao/SG/f2_igfp_optir3_vitcrutin_plb_n4cl_bili_xy22.ome.tif']
{'x': 5012, 'y': 5012, 'z': 6}


In [9]:
# psf647 = tf.imread("PSF/AF647_0.3.tif")
# psfruby =  tf.imread('PSF/mRuby3_0.3.tif')
# psfgfp =  tf.imread('PSF/green_0.3.tif')
# psfdapi =  tf.imread('PSF/DAPI_0.3.tif')
# print(psf647.shape)
# psf647 = psf647[15:-15]
# psfruby = psfruby[15:-15]
# psfgfp = psfgfp[15:-15]
# psfdapi = psfdapi[15:-15]
# print(psfdapi.shape)

psf647 = tf.imread("/home/jmamede/Data/PSF/AF647.tif")
psfruby =  tf.imread('/home/jmamede/Data/PSF/mRuby3.tif')
psfgfp =  tf.imread('/home/jmamede/Data/PSF/green.tif')
psfdapi =  tf.imread('/home/jmamede/Data/PSF/DAPI.tif')
print(psf647.shape)

(23, 256, 256)


In [12]:
# filelist = filelist[2:]
# filelist
# ficheiro = filelist[0]
# frames = pims.bioformats.BioformatsReader(ficheiro,java_memory='1024m')
# print(frames.sizes)

In [13]:
# na = 1.42
# #ra = meta['refractive_index1']
# ra = 1.5156
# z = frames.sizes['z']
# #     x = frames.sizes['x'] // 2
# #     y = frames.sizes['y'] // 2
# #     z = 24
# x = 128
# y = 128
# res_z = xml.image().Pixels.PhysicalSizeZ
# res_xy =  xml.image().Pixels.PhysicalSizeX


# # psf647 = psf647[1:-1]
# # psfruby = psfruby[1:-1]
# # psfgfp = psfgfp[1:-1]
# # psfdapi = psfdapi[1:-1]

# psf = fd_psf.GibsonLanni(
#     na = na, wavelength= 0.625 , size_z=  z, size_x=  x , size_y=  y
#     ,m =60, ns =  1.333
#     , tg =  170, tg0 = 170
#     , ni0 =  ra, ni =  ra
#     ,res_lateral =  res_xy , res_axial = res_z, 
#     #pz = 1
# )
# psfch = psf.generate()

# psf = fd_psf.GibsonLanni(
#     na = na, wavelength= 0.548 , size_z=  z, size_x=  x , size_y=  y
#     ,m =60, ns =  1.333
#     , tg =  170, tg0 = 170
#     , ni0 =  ra, ni =  ra
#     ,res_lateral =  res_xy , res_axial = res_z, 
#     #pz = 1
# )
# psfyfp = psf.generate()



In [14]:

for fname in filelist:
    frames =  pims.bioformats.BioformatsReader(fname,java_memory='1024m')
#     frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default
    # frames.iter_axes
    channels = [frames.metadata.ChannelEmissionWavelength(0,ch) for ch in range(frames.metadata.ChannelCount(0))]
    metadata = frames.metadata
    xml = pimsmeta2OMEXML(frames)
    prjxml = pimsmeta2OMEXML(frames, project=True)

    psf =[]
    #psf = fd_psf.GibsonLanni.load('/tmp/psf.json')

    # args = [na=1.4, wavelength= 0.594 , size_z=  11, size_x=  2048, size_y=  2044,
    #         m :60,ns =  1.333, ni =  1.5156, tg =  170,         
    #         res_lateral =  0.108333333333333, res_axial =  0.3, pZ = 1]

    # args = [{"na": 1.4, "wavelength": '0.594' ,
    #          "size_z": 11, "size_x": 2048, "size_y": 2044,
    #          "m" :60,"ns" : 1.333, "ni" : 1.5156, "tg" : 170,
    #          "res_lateral" : 0.108333333333333, "res_axial" : 0.3,
    #          "pZ" :1}]

    #na = meta['NA']
    z = frames.sizes['z']
    res_z = xml.image().Pixels.PhysicalSizeZ
    res_xy =  xml.image().Pixels.PhysicalSizeX
 
    algo = fd_restoration.RichardsonLucyDeconvolver(n_dims=psfgfp.ndim
                                                    , pad_mode='2357'
#                                                     , pad_mode='none'
#                                                     ,pad_min=(0,1,1)
    #                                                 ,observer_fn=observer
                                                    #,real_domain_fft=True
                                                    #,device='/cpu:0'
                                                   ).initialize()
    # chunk_size=(frames.sizes['z'],511,511)
    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)


    # frames.iter_axes = 'v'
    # for frame in frames:
#     for visit in range(frames.sizes['v']):
    for visit in range(1):
        #%timeit
#         frames.default_coords['v'] = visit
        with tf.TiffWriter(fname[:-3]
#                            +"_v"+str(visit+1)
                           +'.ome.tiff'
                    , bigtiff=True
                    , imagej=False
                          ) as tif:
            with tf.TiffWriter(fname[:-3]
#                            +"_v"+str(visit+1)
                               +'_PRJ.ome.tiff'
                    , bigtiff=True
                    , imagej=False
                          ) as tifprj:
                first = True
    #             print('Visit:',visit)
                for time in range(frames.sizes['t']):
#                 for time in range(1):
    #                 print('Time:',time)
                    res = np.zeros(
                        (frames.sizes['z'],frames.sizes['c']
                         ,frames.sizes['y'],frames.sizes['x'])
    #                      ,2044,2044)
                        , dtype=np.float32)
                    for i in range(frames.sizes['c']):
                        frames.default_coords['c'] = i
    #                     print('Channel:',metadata['channels'][i])
                        ch = str(metadata.ChannelEmissionWavelength(0,i))
        #     ['a647', 'DAPI', 'GFP']
        # ['FITC', 'mRuby3', 'miRFP670']
#         ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
# ['a647', 'DAPI', 'GFP']
                        if ch == 'DAPI' or ch  == '470 nm':
                            psf = psfdapi
                        elif ch == 'miRFP670' or ch == 'a647' or ch == '670.0':
                            psf = psf647
                        elif ch == 'GFP' or ch == 'FITC' or ch == ' 535.0':
                            psf = psfgfp
                        elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == '620.0':
                            psf = psfruby
                        elif ch == 'FRET-gYFP-dsRED':
                            psf = psfruby
                        elif ch == '548.0':
                            psf = psfyfp
                        elif ch == '625.0' or ch == '626.0':
                            psf = psfch
    #                     print(frames[time].shape)
                        arr = da.from_array(frames[time]
    #                                         [:,0:2044,0:2044]
        #                                     [:,:,:]              
                                            , chunks=chunk_size)
    #                     print(frames[time].shape,i, frames[i].max())

        #make mem efficient by creating and empty array with the right shape with all channels and then add it as it goes.
                        res[:,i,:,:] = arr.map_overlap(
        #                 res = arr.map_overlap(
                                deconv
                                ,depth=depthdivide
#                                 ,depth=(0,32,32)
                                ,boundary='reflect'
                                ,dtype='float32').compute(num_workers=1)
            #                     res[i,:,:,:] = arr.map_blocks(
            #                         deconv,dtype='float32').compute(num_workers=1)

    #                     res = da.overlap.trim_internal(res, {0, 16:-16, 16: -16})
        # If we need to swap Channel and Z in the future
        #             img5d = np.swapaxes(img5d,0,1)

                        update_progress( (visit+1) * (time+1) / 1* frames.sizes['t'])
                        print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                                visit+1, 1
#                             frames.sizes['v']
                                
                                ,time+1, frames.sizes['t']
                                , i+1, frames.sizes['c']))
                
#                         update_progress( (visit+1) * (time+1) / (frames.sizes['v']* frames.sizes['t']) )
#                         print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
#                                 visit+1, frames.sizes['v']
#                                 ,time+1, frames.sizes['t']
#                                 , i+1, frames.sizes['c']))


                    if first:
#                         tif.save(res.astype(np.uint16)
# #                             , compress='LZMA'
#                             , description = xml.to_xml()
#                             , photometric='minisblack'
#                             #, datetime= True
#                             , metadata= None
#                             , contiguous=False
#                             )
                        tifprj.save(res.astype(np.uint16).max(axis=0)
#                             , compress='LZMA'
                            , description = prjxml.to_xml()
                            , photometric='minisblack'
                            #, datetime= True
                            , metadata= None
                            , contiguous=False
                            )
                        first = False
                    else:
#                         tif.save(res.astype(np.uint16)
# #                             , compress='LZMA'
#     #                         , description = xml.to_xml()
#                             , photometric='minisblack'
#                             , metadata= None
#                             , contiguous=False
#                             )
                        tifprj.save(res.astype(np.uint16).max(axis=0)
#                             , compress='LZMA'
    #                         , description = xml.to_xml()
                            , photometric='minisblack'
                            #, datetime= True
                            , metadata= None
                            , contiguous=False
                            )

            tif.close()
            tifprj.close()
    update_progress(1)
    
print("Done")

Done


In [ ]:
    
print("Done")